1A. 10 RNN

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

# Sample text
text =("Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting "
       "the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, "
       "including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text. "
       "At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and "
       "predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of "
       "text during the training phase of the model. One of the most popular approaches to next character prediction involves the use of Recurrent "
       "Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for"
       " sequential data like text, as they can maintain information in 'memory' about previous characters to inform the prediction of the next character."
       " LSTM networks enhance this capability by being able to remember long-term dependencies, making them even more effective for next character "
       "prediction tasks. Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the "
       "probability of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters "
       "to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time. Once trained, "
       "the model can be used to predict the next character in a given piece of text by considering the sequence of characters that precede it. "
       "This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion features, and "
       "enable more natural interactions with AI-based chatbots and virtual assistants. In summary, next character prediction plays a crucial role in "
       "enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. Through the use"
       " of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities for the future "
       "of text-based technology.")

chars = sorted(list(set(text)))
ix_to_char = {i: ch for i, ch in enumerate(chars)}
char_to_ix = {ch: i for i, ch in enumerate(chars)} 
chars = sorted(list(set(text)))




# Preparing the dataset
max_length = 10  # Maximum length of input sequences
X = []
y = []
for i in range(len(text) - max_length):
    sequence = text[i:i + max_length]
    label = text[i + max_length]
    X.append([char_to_ix[char] for char in sequence])
    y.append(char_to_ix[label])

X = np.array(X)
y = np.array(y)

# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Converting data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)

# Defining the RNN model
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])  # Get the output of the last RNN cell
        return output

class CharLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.lstm(embedded)
        output = self.fc(output[:, -1, :])
        return output

class CharGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.gru(embedded)
        output = self.fc(output[:, -1, :])
        return output
# Hyperparameters
hidden_size = 128
learning_rate = 0.005
epochs = 100

# Model, loss, and optimizer
model = CharRNN(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Next character prediction is a fundam"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

FileNotFoundError: [Errno 2] No such file or directory: 'tiny_shakespeare.txt'

20 RNN

In [2]:
max_length = 20
# Model, loss, and optimizer
model = CharRNN(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Next character prediction is a fundam"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Epoch 10, Loss: 2.5220096111297607, Validation Loss: 2.4657649993896484, Validation Accuracy: 0.3340336084365845
Epoch 20, Loss: 2.031263589859009, Validation Loss: 2.1571600437164307, Validation Accuracy: 0.3949579894542694
Epoch 30, Loss: 1.6530259847640991, Validation Loss: 2.0119218826293945, Validation Accuracy: 0.43907561898231506
Epoch 40, Loss: 1.302440881729126, Validation Loss: 1.9274914264678955, Validation Accuracy: 0.4768907427787781
Epoch 50, Loss: 0.9809191823005676, Validation Loss: 1.8938294649124146, Validation Accuracy: 0.4978991448879242
Epoch 60, Loss: 0.6974117159843445, Validation Loss: 1.9417282342910767, Validation Accuracy: 0.4957983195781708
Epoch 70, Loss: 0.45580536127090454, Validation Loss: 1.992605209350586, Validation Accuracy: 0.4957983195781708
Epoch 80, Loss: 0.2755407989025116, Validation Loss: 2.0931529998779297, Validation Accuracy: 0.4852941036224365
Epoch 90, Loss: 0.16291145980358124, Validation Loss: 2.197254180908203, Validation Accuracy: 0.4

30 RNN

In [3]:
max_length = 30
# Model, loss, and optimizer
model = CharRNN(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Next character prediction is a fundam"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Epoch 10, Loss: 2.5338222980499268, Validation Loss: 2.468808650970459, Validation Accuracy: 0.3172268867492676
Epoch 20, Loss: 2.047011375427246, Validation Loss: 2.1640560626983643, Validation Accuracy: 0.4264705777168274
Epoch 30, Loss: 1.6739567518234253, Validation Loss: 1.994498372077942, Validation Accuracy: 0.47268906235694885
Epoch 40, Loss: 1.3376351594924927, Validation Loss: 1.914099097251892, Validation Accuracy: 0.5210084319114685
Epoch 50, Loss: 1.0287061929702759, Validation Loss: 1.8982856273651123, Validation Accuracy: 0.5336134433746338
Epoch 60, Loss: 0.7455872297286987, Validation Loss: 1.9173685312271118, Validation Accuracy: 0.5231092572212219
Epoch 70, Loss: 0.49708855152130127, Validation Loss: 2.003119707107544, Validation Accuracy: 0.5084033608436584
Epoch 80, Loss: 0.30718210339546204, Validation Loss: 2.1226227283477783, Validation Accuracy: 0.5147058963775635
Epoch 90, Loss: 0.18527762591838837, Validation Loss: 2.219557046890259, Validation Accuracy: 0.51

1O LSTM

In [4]:
max_length = 10
# Model, loss, and optimizer
model = CharLSTM(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Next character prediction is a fundam"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Epoch 10, Loss: 2.520794630050659, Validation Loss: 2.4925127029418945, Validation Accuracy: 0.34663864970207214
Epoch 20, Loss: 2.016547203063965, Validation Loss: 2.1818032264709473, Validation Accuracy: 0.43907561898231506
Epoch 30, Loss: 1.6090964078903198, Validation Loss: 2.009507417678833, Validation Accuracy: 0.45168066024780273
Epoch 40, Loss: 1.2487033605575562, Validation Loss: 1.915102481842041, Validation Accuracy: 0.4978991448879242
Epoch 50, Loss: 0.9222983121871948, Validation Loss: 1.8958710432052612, Validation Accuracy: 0.5126050710678101
Epoch 60, Loss: 0.6343286037445068, Validation Loss: 1.9344927072525024, Validation Accuracy: 0.5210084319114685
Epoch 70, Loss: 0.4052569568157196, Validation Loss: 2.006352663040161, Validation Accuracy: 0.5126050710678101
Epoch 80, Loss: 0.2449502795934677, Validation Loss: 2.1200826168060303, Validation Accuracy: 0.5105041861534119
Epoch 90, Loss: 0.1507578194141388, Validation Loss: 2.216162919998169, Validation Accuracy: 0.516

20 LSTM

In [5]:
max_length = 20
# Model, loss, and optimizer
model = CharLSTM(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Next character prediction is a fundam"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Epoch 10, Loss: 2.549938440322876, Validation Loss: 2.504688024520874, Validation Accuracy: 0.3445378243923187
Epoch 20, Loss: 2.032592535018921, Validation Loss: 2.1786019802093506, Validation Accuracy: 0.424369752407074
Epoch 30, Loss: 1.6261953115463257, Validation Loss: 1.9955672025680542, Validation Accuracy: 0.5042017102241516
Epoch 40, Loss: 1.2634762525558472, Validation Loss: 1.913388967514038, Validation Accuracy: 0.5105041861534119
Epoch 50, Loss: 0.9457004070281982, Validation Loss: 1.9174299240112305, Validation Accuracy: 0.5210084319114685
Epoch 60, Loss: 0.6652361154556274, Validation Loss: 1.9478936195373535, Validation Accuracy: 0.5252100825309753
Epoch 70, Loss: 0.43496087193489075, Validation Loss: 2.0106894969940186, Validation Accuracy: 0.5273109078407288
Epoch 80, Loss: 0.2653399407863617, Validation Loss: 2.1706900596618652, Validation Accuracy: 0.506302535533905
Epoch 90, Loss: 0.1608719676733017, Validation Loss: 2.318385362625122, Validation Accuracy: 0.495798

30 LSTM

In [6]:
max_length = 30
# Model, loss, and optimizer
model = CharLSTM(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Next character prediction is a fundam"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Epoch 10, Loss: 2.5018138885498047, Validation Loss: 2.4373841285705566, Validation Accuracy: 0.3445378243923187
Epoch 20, Loss: 2.00844144821167, Validation Loss: 2.1338062286376953, Validation Accuracy: 0.4306722581386566
Epoch 30, Loss: 1.6190096139907837, Validation Loss: 1.9776421785354614, Validation Accuracy: 0.4474789798259735
Epoch 40, Loss: 1.2626203298568726, Validation Loss: 1.8995037078857422, Validation Accuracy: 0.4852941036224365
Epoch 50, Loss: 0.9395723938941956, Validation Loss: 1.906023383140564, Validation Accuracy: 0.506302535533905
Epoch 60, Loss: 0.6515686511993408, Validation Loss: 1.9822111129760742, Validation Accuracy: 0.5168067216873169
Epoch 70, Loss: 0.41137897968292236, Validation Loss: 2.0669515132904053, Validation Accuracy: 0.48949578404426575
Epoch 80, Loss: 0.24650317430496216, Validation Loss: 2.1952030658721924, Validation Accuracy: 0.4810924232006073
Epoch 90, Loss: 0.14683128893375397, Validation Loss: 2.3495113849639893, Validation Accuracy: 0.

10 GRU

In [7]:
max_length = 10
# Model, loss, and optimizer
model = CharGRU(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Next character prediction is a fundam"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Epoch 10, Loss: 2.395632266998291, Validation Loss: 2.400958299636841, Validation Accuracy: 0.3424369692802429
Epoch 20, Loss: 1.8846566677093506, Validation Loss: 2.089996337890625, Validation Accuracy: 0.4432772994041443
Epoch 30, Loss: 1.4680081605911255, Validation Loss: 1.9066189527511597, Validation Accuracy: 0.5042017102241516
Epoch 40, Loss: 1.0945087671279907, Validation Loss: 1.8267385959625244, Validation Accuracy: 0.5231092572212219
Epoch 50, Loss: 0.7577113509178162, Validation Loss: 1.8545242547988892, Validation Accuracy: 0.5273109078407288
Epoch 60, Loss: 0.4758239686489105, Validation Loss: 1.945866584777832, Validation Accuracy: 0.529411792755127
Epoch 70, Loss: 0.2701752781867981, Validation Loss: 2.0990848541259766, Validation Accuracy: 0.5147058963775635
Epoch 80, Loss: 0.14842531085014343, Validation Loss: 2.2591664791107178, Validation Accuracy: 0.5126050710678101
Epoch 90, Loss: 0.0918559655547142, Validation Loss: 2.3851773738861084, Validation Accuracy: 0.5
Ep

20 GRU

In [8]:
max_length = 20
# Model, loss, and optimizer
model = CharGRU(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Next character prediction is a fundam"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Epoch 10, Loss: 2.3771276473999023, Validation Loss: 2.3742356300354004, Validation Accuracy: 0.36764705181121826
Epoch 20, Loss: 1.8542933464050293, Validation Loss: 2.0918853282928467, Validation Accuracy: 0.43487393856048584
Epoch 30, Loss: 1.437756896018982, Validation Loss: 1.9142112731933594, Validation Accuracy: 0.5042017102241516
Epoch 40, Loss: 1.054316759109497, Validation Loss: 1.8693901300430298, Validation Accuracy: 0.5105041861534119
Epoch 50, Loss: 0.7123831510543823, Validation Loss: 1.906084418296814, Validation Accuracy: 0.5336134433746338
Epoch 60, Loss: 0.4355407655239105, Validation Loss: 2.002800941467285, Validation Accuracy: 0.5336134433746338
Epoch 70, Loss: 0.2421967089176178, Validation Loss: 2.148542881011963, Validation Accuracy: 0.5189075469970703
Epoch 80, Loss: 0.1324925720691681, Validation Loss: 2.299184560775757, Validation Accuracy: 0.5084033608436584
Epoch 90, Loss: 0.08318573236465454, Validation Loss: 2.4269232749938965, Validation Accuracy: 0.531

30 GRU

In [9]:
max_length = 30
# Model, loss, and optimizer
model = CharGRU(len(chars), hidden_size, len(chars))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, y_val)
        _, predicted = torch.max(val_output, 1)
        val_accuracy = (predicted == y_val).float().mean()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "Next character prediction is a fundam"
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")

Epoch 10, Loss: 2.3970115184783936, Validation Loss: 2.3555400371551514, Validation Accuracy: 0.3571428656578064
Epoch 20, Loss: 1.8836690187454224, Validation Loss: 2.068542003631592, Validation Accuracy: 0.45168066024780273
Epoch 30, Loss: 1.4683642387390137, Validation Loss: 1.8898626565933228, Validation Accuracy: 0.48739495873451233
Epoch 40, Loss: 1.0928021669387817, Validation Loss: 1.8155921697616577, Validation Accuracy: 0.5231092572212219
Epoch 50, Loss: 0.7567638754844666, Validation Loss: 1.8360581398010254, Validation Accuracy: 0.5462185144424438
Epoch 60, Loss: 0.4811756908893585, Validation Loss: 1.9101457595825195, Validation Accuracy: 0.5420168042182922
Epoch 70, Loss: 0.28263720870018005, Validation Loss: 2.027892827987671, Validation Accuracy: 0.5315126180648804
Epoch 80, Loss: 0.15859577059745789, Validation Loss: 2.160409450531006, Validation Accuracy: 0.5357142686843872
Epoch 90, Loss: 0.09645137935876846, Validation Loss: 2.2762715816497803, Validation Accuracy: 

2 20

In [23]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import requests
import time




# Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text

# Character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

# Dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# LSTM Model class
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(vocab_size, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_dim)
        c0 = torch.zeros(num_layers, x.size(0), hidden_dim)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Function to train and validate the model
def train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0

        for sequences, targets in train_loader:
            sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
            optimizer.zero_grad()
            outputs = model(sequences_one_hot)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train_correct += (predicted == targets).sum().item()
            total_train_samples += targets.size(0)

        train_accuracy = 100 * total_train_correct / total_train_samples

        model.eval()
        total_val_loss = 0
        total_val_correct = 0
        total_val_samples = 0
        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
                outputs = model(sequences_one_hot)
                loss = criterion(outputs, targets)
                total_val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val_correct += (predicted == targets).sum().item()
                total_val_samples += targets.size(0)

        val_accuracy = 100 * total_val_correct / total_val_samples

        print(f'Epoch {epoch+1}, Train Loss: {total_train_loss / len(train_loader)}, '
              f'Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {total_val_loss / len(test_loader)}, '
              f'Val Accuracy: {val_accuracy:.2f}%')

    end_time = time.time()  # Record the end time of training
    training_time = end_time - start_time  # Calculate the total training time
    print(f'Total Training Time: {training_time:.2f} seconds')

# Prediction function
def predict_next_char(model, char_to_int, int_to_char, initial_str, max_length):
    model.eval()
    with torch.no_grad():
        sequence = [char_to_int.get(c, 0) for c in initial_str[-max_length:]]  # Handle characters not in dict
        sequence = torch.tensor(sequence, dtype=torch.long).unsqueeze(0)
        sequence_one_hot = torch.nn.functional.one_hot(sequence, num_classes=vocab_size).float()
        prediction = model(sequence_one_hot)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return int_to_char[predicted_index]

# Main script
if __name__ == "__main__":
    sequence_length = 20  # Choose a sequence length
    hidden_dim = 128  # Choose a hidden dimension size
    num_layers = 2  # Choose number of LSTM layers
    batch_size = 128  # Batch size for training
    epochs = 5  # Number of epochs to train for

    # Prepare the dataset
    sequences, targets = [], []
    for i in range(0, len(text) - sequence_length):
        seq = [char_to_int[ch] for ch in text[i:i+sequence_length]]
        target = char_to_int[text[i+sequence_length]]
        sequences.append(seq)
        targets.append(target)

    sequences = torch.tensor(sequences, dtype=torch.long)
    targets = torch.tensor(targets, dtype=torch.long)

    dataset = CharDataset(sequences, targets)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    model = LSTMModel(vocab_size, hidden_dim, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Train and validate the model
    train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=epochs)

    # Prediction example after training
    test_str = "We are accounted poor citiz"
    predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str, sequence_length)
    print(f"Predicted next character: '{predicted_char}'")


Epoch 1, Train Loss: 1.7511989493446656, Train Accuracy: 48.15%, Val Loss: 1.5606276364553542, Val Accuracy: 52.76%
Epoch 2, Train Loss: 1.5173584691628736, Train Accuracy: 53.95%, Val Loss: 1.4977184385659705, Val Accuracy: 54.61%
Epoch 3, Train Loss: 1.4786085673980307, Train Accuracy: 54.92%, Val Loss: 1.4776417698862905, Val Accuracy: 54.99%
Epoch 4, Train Loss: 1.4576729451932924, Train Accuracy: 55.33%, Val Loss: 1.4668254681312005, Val Accuracy: 54.96%
Epoch 5, Train Loss: 1.444161927946258, Train Accuracy: 55.67%, Val Loss: 1.450158799782062, Val Accuracy: 55.70%
Total Training Time: 978.80 seconds
Predicted next character: 'e'


In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import requests
import time

# Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text

# Character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

# Dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# LSTM Model class
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(vocab_size, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_dim)
        c0 = torch.zeros(num_layers, x.size(0), hidden_dim)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Function to train and validate the model
def train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0

        for sequences, targets in train_loader:
            sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
            optimizer.zero_grad()
            outputs = model(sequences_one_hot)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train_correct += (predicted == targets).sum().item()
            total_train_samples += targets.size(0)

        train_accuracy = 100 * total_train_correct / total_train_samples

        model.eval()
        total_val_loss = 0
        total_val_correct = 0
        total_val_samples = 0
        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
                outputs = model(sequences_one_hot)
                loss = criterion(outputs, targets)
                total_val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val_correct += (predicted == targets).sum().item()
                total_val_samples += targets.size(0)

        val_accuracy = 100 * total_val_correct / total_val_samples

        print(f'Epoch {epoch+1}, Train Loss: {total_train_loss / len(train_loader)}, '
              f'Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {total_val_loss / len(test_loader)}, '
              f'Val Accuracy: {val_accuracy:.2f}%')

    end_time = time.time()  # Record the end time of training
    training_time = end_time - start_time  # Calculate the total training time
    print(f'Total Training Time: {training_time:.2f} seconds')

# Prediction function
def predict_next_char(model, char_to_int, int_to_char, initial_str, max_length):
    model.eval()
    with torch.no_grad():
        sequence = [char_to_int.get(c, 0) for c in initial_str[-max_length:]]  # Handle characters not in dict
        sequence = torch.tensor(sequence, dtype=torch.long).unsqueeze(0)
        sequence_one_hot = torch.nn.functional.one_hot(sequence, num_classes=vocab_size).float()
        prediction = model(sequence_one_hot)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return int_to_char[predicted_index]

# Main script
if __name__ == "__main__":
    sequence_length = 30  # Choose a sequence length
    hidden_dim = 128  # Choose a hidden dimension size
    num_layers = 2  # Choose number of LSTM layers
    batch_size = 128  # Batch size for training
    epochs = 5  # Number of epochs to train for

    # Prepare the dataset
    sequences, targets = [], []
    for i in range(0, len(text) - sequence_length):
        seq = [char_to_int[ch] for ch in text[i:i+sequence_length]]
        target = char_to_int[text[i+sequence_length]]
        sequences.append(seq)
        targets.append(target)

    sequences = torch.tensor(sequences, dtype=torch.long)
    targets = torch.tensor(targets, dtype=torch.long)

    dataset = CharDataset(sequences, targets)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    model = LSTMModel(vocab_size, hidden_dim, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Train and validate the model
    train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=epochs)

    # Prediction example after training
    test_str = "We are accounted poor citiz"
    predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str, sequence_length)
    print(f"Predicted next character: '{predicted_char}'")


Epoch 1, Train Loss: 1.6962894519392404, Train Accuracy: 49.58%, Val Loss: 1.534514790899919, Val Accuracy: 53.48%
Epoch 2, Train Loss: 1.4898458307868643, Train Accuracy: 54.68%, Val Loss: 1.4744970922967995, Val Accuracy: 54.98%
Epoch 3, Train Loss: 1.4509233574859042, Train Accuracy: 55.65%, Val Loss: 1.4517270606000465, Val Accuracy: 55.58%
Epoch 4, Train Loss: 1.428999951112496, Train Accuracy: 56.17%, Val Loss: 1.44063881013972, Val Accuracy: 55.91%
Epoch 5, Train Loss: 1.4140226065052424, Train Accuracy: 56.55%, Val Loss: 1.430225047139142, Val Accuracy: 56.32%
Total Training Time: 1531.09 seconds
Predicted next character: 'e'


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import requests
import time

# Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text

# Character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

# Dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# LSTM Model class
class GRUModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super(GRUModel, self).__init__()
        self.lstm = nn.GRU(vocab_size, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_dim)
        out, _ = self.lstm(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Function to train and validate the model
def train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0

        for sequences, targets in train_loader:
            sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
            optimizer.zero_grad()
            outputs = model(sequences_one_hot)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train_correct += (predicted == targets).sum().item()
            total_train_samples += targets.size(0)

        train_accuracy = 100 * total_train_correct / total_train_samples

        model.eval()
        total_val_loss = 0
        total_val_correct = 0
        total_val_samples = 0
        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
                outputs = model(sequences_one_hot)
                loss = criterion(outputs, targets)
                total_val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val_correct += (predicted == targets).sum().item()
                total_val_samples += targets.size(0)

        val_accuracy = 100 * total_val_correct / total_val_samples

        print(f'Epoch {epoch+1}, Train Loss: {total_train_loss / len(train_loader)}, '
              f'Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {total_val_loss / len(test_loader)}, '
              f'Val Accuracy: {val_accuracy:.2f}%')

    end_time = time.time()  # Record the end time of training
    training_time = end_time - start_time  # Calculate the total training time
    print(f'Total Training Time: {training_time:.2f} seconds')

# Prediction function
def predict_next_char(model, char_to_int, int_to_char, initial_str, max_length):
    model.eval()
    with torch.no_grad():
        sequence = [char_to_int.get(c, 0) for c in initial_str[-max_length:]]  # Handle characters not in dict
        sequence = torch.tensor(sequence, dtype=torch.long).unsqueeze(0)
        sequence_one_hot = torch.nn.functional.one_hot(sequence, num_classes=vocab_size).float()
        prediction = model(sequence_one_hot)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return int_to_char[predicted_index]

# Main script
if __name__ == "__main__":
    sequence_length = 20  # Choose a sequence length
    hidden_dim = 128  # Choose a hidden dimension size
    num_layers = 2  # Choose number of LSTM layers
    batch_size = 128  # Batch size for training
    epochs = 5  # Number of epochs to train for

    # Prepare the dataset
    sequences, targets = [], []
    for i in range(0, len(text) - sequence_length):
        seq = [char_to_int[ch] for ch in text[i:i+sequence_length]]
        target = char_to_int[text[i+sequence_length]]
        sequences.append(seq)
        targets.append(target)

    sequences = torch.tensor(sequences, dtype=torch.long)
    targets = torch.tensor(targets, dtype=torch.long)

    dataset = CharDataset(sequences, targets)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    model = GRUModel(vocab_size, hidden_dim, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Train and validate the model
    train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=epochs)

    # Prediction example after training
    test_str = "We are accounted poor citiz"
    predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str, sequence_length)
    print(f"Predicted next character: '{predicted_char}'")


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import requests
import time

# Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text

# Character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

# Dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# LSTM Model class
class GRUModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super(GRUModel, self).__init__()
        self.lstm = nn.GRU(vocab_size, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_dim)
        out, _ = self.lstm(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Function to train and validate the model
def train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0

        for sequences, targets in train_loader:
            sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
            optimizer.zero_grad()
            outputs = model(sequences_one_hot)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train_correct += (predicted == targets).sum().item()
            total_train_samples += targets.size(0)

        train_accuracy = 100 * total_train_correct / total_train_samples

        model.eval()
        total_val_loss = 0
        total_val_correct = 0
        total_val_samples = 0
        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
                outputs = model(sequences_one_hot)
                loss = criterion(outputs, targets)
                total_val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val_correct += (predicted == targets).sum().item()
                total_val_samples += targets.size(0)

        val_accuracy = 100 * total_val_correct / total_val_samples

        print(f'Epoch {epoch+1}, Train Loss: {total_train_loss / len(train_loader)}, '
              f'Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {total_val_loss / len(test_loader)}, '
              f'Val Accuracy: {val_accuracy:.2f}%')

    end_time = time.time()  # Record the end time of training
    training_time = end_time - start_time  # Calculate the total training time
    print(f'Total Training Time: {training_time:.2f} seconds')

# Prediction function
def predict_next_char(model, char_to_int, int_to_char, initial_str, max_length):
    model.eval()
    with torch.no_grad():
        sequence = [char_to_int.get(c, 0) for c in initial_str[-max_length:]]  # Handle characters not in dict
        sequence = torch.tensor(sequence, dtype=torch.long).unsqueeze(0)
        sequence_one_hot = torch.nn.functional.one_hot(sequence, num_classes=vocab_size).float()
        prediction = model(sequence_one_hot)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return int_to_char[predicted_index]

# Main script
if __name__ == "__main__":
    sequence_length = 30  # Choose a sequence length
    hidden_dim = 128  # Choose a hidden dimension size
    num_layers = 2  # Choose number of LSTM layers
    batch_size = 128  # Batch size for training
    epochs = 5  # Number of epochs to train for

    # Prepare the dataset
    sequences, targets = [], []
    for i in range(0, len(text) - sequence_length):
        seq = [char_to_int[ch] for ch in text[i:i+sequence_length]]
        target = char_to_int[text[i+sequence_length]]
        sequences.append(seq)
        targets.append(target)

    sequences = torch.tensor(sequences, dtype=torch.long)
    targets = torch.tensor(targets, dtype=torch.long)

    dataset = CharDataset(sequences, targets)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    model = GRUModel(vocab_size, hidden_dim, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Train and validate the model
    train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=epochs)

    # Prediction example after training
    test_str = "We are accounted poor citiz"
    predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str, sequence_length)
    print(f"Predicted next character: '{predicted_char}'")


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import requests
import time

# Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text

# Character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

# Dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# LSTM Model class
class GRUModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super(GRUModel, self).__init__()
        self.lstm = nn.GRU(vocab_size, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_dim)
        out, _ = self.lstm(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Function to train and validate the model
def train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0

        for sequences, targets in train_loader:
            sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
            optimizer.zero_grad()
            outputs = model(sequences_one_hot)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train_correct += (predicted == targets).sum().item()
            total_train_samples += targets.size(0)

        train_accuracy = 100 * total_train_correct / total_train_samples

        model.eval()
        total_val_loss = 0
        total_val_correct = 0
        total_val_samples = 0
        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
                outputs = model(sequences_one_hot)
                loss = criterion(outputs, targets)
                total_val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val_correct += (predicted == targets).sum().item()
                total_val_samples += targets.size(0)

        val_accuracy = 100 * total_val_correct / total_val_samples

        print(f'Epoch {epoch+1}, Train Loss: {total_train_loss / len(train_loader)}, '
              f'Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {total_val_loss / len(test_loader)}, '
              f'Val Accuracy: {val_accuracy:.2f}%')

    end_time = time.time()  # Record the end time of training
    training_time = end_time - start_time  # Calculate the total training time
    print(f'Total Training Time: {training_time:.2f} seconds')

# Prediction function
def predict_next_char(model, char_to_int, int_to_char, initial_str, max_length):
    model.eval()
    with torch.no_grad():
        sequence = [char_to_int.get(c, 0) for c in initial_str[-max_length:]]  # Handle characters not in dict
        sequence = torch.tensor(sequence, dtype=torch.long).unsqueeze(0)
        sequence_one_hot = torch.nn.functional.one_hot(sequence, num_classes=vocab_size).float()
        prediction = model(sequence_one_hot)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return int_to_char[predicted_index]

# Main script
if __name__ == "__main__":
    sequence_length = 50  # Choose a sequence length
    hidden_dim = 128  # Choose a hidden dimension size
    num_layers = 2  # Choose number of LSTM layers
    batch_size = 128  # Batch size for training
    epochs = 5  # Number of epochs to train for

    # Prepare the dataset
    sequences, targets = [], []
    for i in range(0, len(text) - sequence_length):
        seq = [char_to_int[ch] for ch in text[i:i+sequence_length]]
        target = char_to_int[text[i+sequence_length]]
        sequences.append(seq)
        targets.append(target)

    sequences = torch.tensor(sequences, dtype=torch.long)
    targets = torch.tensor(targets, dtype=torch.long)

    dataset = CharDataset(sequences, targets)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    model = GRUModel(vocab_size, hidden_dim, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Train and validate the model
    train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=epochs)

    # Prediction example after training
    test_str = "We are accounted poor citiz"
    predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str, sequence_length)
    print(f"Predicted next character: '{predicted_char}'")


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import requests
import time

# Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text

# Character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}
vocab_size = len(chars)

# Dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# LSTM Model class
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(vocab_size, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_dim)
        c0 = torch.zeros(num_layers, x.size(0), hidden_dim)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Function to train and validate the model
def train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=5):
    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        total_train_correct = 0
        total_train_samples = 0

        for sequences, targets in train_loader:
            sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
            optimizer.zero_grad()
            outputs = model(sequences_one_hot)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train_correct += (predicted == targets).sum().item()
            total_train_samples += targets.size(0)

        train_accuracy = 100 * total_train_correct / total_train_samples

        model.eval()
        total_val_loss = 0
        total_val_correct = 0
        total_val_samples = 0
        with torch.no_grad():
            for sequences, targets in test_loader:
                sequences_one_hot = torch.nn.functional.one_hot(sequences, num_classes=vocab_size).float()
                outputs = model(sequences_one_hot)
                loss = criterion(outputs, targets)
                total_val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val_correct += (predicted == targets).sum().item()
                total_val_samples += targets.size(0)

        val_accuracy = 100 * total_val_correct / total_val_samples

        print(f'Epoch {epoch+1}, Train Loss: {total_train_loss / len(train_loader)}, '
              f'Train Accuracy: {train_accuracy:.2f}%, '
              f'Val Loss: {total_val_loss / len(test_loader)}, '
              f'Val Accuracy: {val_accuracy:.2f}%')

    end_time = time.time()  # Record the end time of training
    training_time = end_time - start_time  # Calculate the total training time
    print(f'Total Training Time: {training_time:.2f} seconds')

# Prediction function
def predict_next_char(model, char_to_int, int_to_char, initial_str, max_length):
    model.eval()
    with torch.no_grad():
        sequence = [char_to_int.get(c, 0) for c in initial_str[-max_length:]]  # Handle characters not in dict
        sequence = torch.tensor(sequence, dtype=torch.long).unsqueeze(0)
        sequence_one_hot = torch.nn.functional.one_hot(sequence, num_classes=vocab_size).float()
        prediction = model(sequence_one_hot)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return int_to_char[predicted_index]

# Main script
if __name__ == "__main__":
    sequence_length = 50  # Choose a sequence length
    hidden_dim = 128  # Choose a hidden dimension size
    num_layers = 2  # Choose number of LSTM layers
    batch_size = 128  # Batch size for training
    epochs = 5  # Number of epochs to train for

    # Prepare the dataset
    sequences, targets = [], []
    for i in range(0, len(text) - sequence_length):
        seq = [char_to_int[ch] for ch in text[i:i+sequence_length]]
        target = char_to_int[text[i+sequence_length]]
        sequences.append(seq)
        targets.append(target)

    sequences = torch.tensor(sequences, dtype=torch.long)
    targets = torch.tensor(targets, dtype=torch.long)

    dataset = CharDataset(sequences, targets)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Initialize the model
    model = LSTMModel(vocab_size, hidden_dim, num_layers)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # Train and validate the model
    train_and_validate(model, train_loader, test_loader, criterion, optimizer, epochs=epochs)

    # Prediction example after training
    test_str = "We are accounted poor citiz"
    predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str, sequence_length)
    print(f"Predicted next character: '{predicted_char}'")
